<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Diverse Plugins for Decison Making

Decision Intelligence applied in this module:  
* Using the power of Collective Intelligence (wisdom & information from multiple experts) 
* Using data statistics & analytics as source information for a decision   
* Using Probability as a quantitative approach to validate decisions with Machine Learning  

> "Probability is the language of uncertainty, and it's the only language we have for communicating what we know and don't know."
>
> -- <cite>Nate Silver (famous statistician and super-forecaster)</cite> 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets
* Use the API configuration to build the Semantic Kernel orchestrator
* The configuration builder will retrieve the Bing Search API Key

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.20.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.20.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.20.0-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"
#r "nuget: Microsoft.ML, 3.0.1"
#r "nuget: Microsoft.ML.FastTree, 3.0.1"
#r "nuget: Microsoft.Extensions.ML, 3.0.1"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Bing.WebSearch;
using System.ComponentModel;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.Extensions.ML, 3.0.1 Microsoft.ML, 3.0.1 Microsoft.ML.FastTree, 3.0.1 Microsoft.SemanticKernel, 1.20.0 Microsoft.SemanticKernel.Plugins.Core, 1.20.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.20.0-alpha

In [2]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

// Import the BaseballHallOfFamePlugin.cs file
#!import ../Classes/BaseballHallOfFamePlugin.cs

// Add the native plugin with the three native functions to the semantic kernel
semanticKernel.ImportPluginFromType<BaseballHallOfFamePlugin>();

Using Azure OpenAI Service


### Step 2 - Introduce Sports Decision Scenario 

<img width ="1024x" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SportsDecisionScenario.png">

### Step 3 - Background Information

Execute the cell below to understand what type of collective information will be used to help the GenAI model make the decsion. In addition to the internet (knowledge graph) shown in previous modules, historical baseball statistic and Machine Learning models exposing a quantitative probability are available.  

In [3]:
// Background Information on what is happening & components

// 1) Directory of the historical baseball data statistics
string filePathMLBBaseballBatters = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Data/MLBBaseballBattersPositionPlayers.csv";
// Read the batters data from the CSV file
var batters = File.ReadAllLines(filePathMLBBaseballBatters)
                        .Skip(1)
                        .Select(v => MLBBaseballBatter.FromCsv(v))
                        .ToList();

// 2) A custom decision rules engine crafted by a Machine Learning model
// Machine Learing Model and Prediction Engine                       
string modelPath = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Models/InductedToHoF-GeneralizedAdditiveModels.mlnet";
var mlContext = new MLContext(seed: 100);
ITransformer loadedModel;
DataViewSchema schema;

using (var stream = File.OpenRead(modelPath))
{
    loadedModel = mlContext.Model.Load(stream, out schema);
}
TransformerChain<ITransformer> transfomerChain = (TransformerChain<ITransformer>)loadedModel;

var predEngineInductedToHallOfFame = mlContext.Model.CreatePredictionEngine<MLBBaseballBatter, MLBHOFPrediction>(transfomerChain);

// 3) Predict if Mike Trout will be inducted to the Hall of Fame
var mikeTroutStats = batters.Where(b => b.FullPlayerName == "Mike Trout").FirstOrDefault();
Console.WriteLine("Mike Trout Stats:");
Console.WriteLine(mikeTroutStats.ToString());
var prediction = predEngineInductedToHallOfFame.Predict(mikeTroutStats);
Console.WriteLine("--------------------");
Console.WriteLine("Mike Trout Probabibitiy of being inducted to Hall Of Fame:");
Console.WriteLine(prediction.Probability);

Mike Trout Stats:
FullPlayerName:Mike Trout, YearPlayed:12, AB:5094
        R:1052, H:1543, Doubles:296, Triples:51, HR:350, RBI:896, SB:204,
        BattingAvg:0.303, Slg:0.587, AllStar:9,
        TB:2991, LastYearPlayed:2022, ID:troutmi01
--------------------
Mike Trout Probabibitiy of being inducted to Hall Of Fame:
0.69909585


### Step 4 - Make Hall of Fame Decision - Only using Web Search Results 

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame. **Notice the instruction in the prompt is to only use the web search results.** While there might be multiple functions available to the GenAI model via the imported plugin, the instruction is to use only the single "tool" at it's disposal.  

In [4]:
// Use WebSearch to find information on Mike Trout

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Only use web search results to make your decision.
Provide sources of information and explain your reasoning.
Ensure to cite all the sources using [number] notation of each URL after the reference in order.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

To evaluate whether Mike Trout should be inducted into the Hall of Fame, we can analyze several key factors based on current web sources.

### 1. **Career Longevity and Performance**
Mike Trout is considered one of the most talented players in baseball history. As of now, he has met the Hall of Fame eligibility criteria by playing the necessary ten seasons in the majors [1]. However, his recent injury struggles have raised concerns. Since 2016, Trout has only played more than 140 games in one season, and he has missed a significant number of games due to injuries over the past few years [5][6]. By the end of 2024, he will have played in only 41% of possible games since 2021, which could detract from his overall statistics and legacy [7].

### 2. **Impact of Injuries**
Trout's injuries have been a major topic of discussion regarding his Hall of Fame candidacy. Each lost season impacts his cumulative statistics, which are a critical part of Hall of Fame considerations [3][5]. While his p

### Step 5 - Make Hall of Fame Decision - Only using Baseball Statistics 

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame. **Notice the instruction in the prompt is to only use the historical baseball statistics.** While there might be multiple functions available to the GenAI model via the imported plugin, the instruction is to use only the single "tool" at it's disposal.  

In [5]:
// Use Baseball statistics to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Only use the historical baseball statistics to make your decision. 
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

To determine if Mike Trout should be inducted into the Hall of Fame based solely on historical baseball statistics, we would typically consider several key metrics:

1. **Career Longevity and Years Played**: A player’s longevity in the league can impact their statistics and overall legacy.

2. **Batting Average (BA)**: Indicates a player’s hitting ability.

3. **Home Runs (HR)**: A significant measure of a player's power-hitting capability.

4. **Runs Batted In (RBI)**: Reflects a player's ability to drive in runs.

5. **On-Base Percentage (OBP) and Slugging Percentage (SLG)**: These metrics provide insight into a player's overall offensive contributions.

6. **Wins Above Replacement (WAR)**: A comprehensive statistic that estimates a player’s overall value to their team.

7. **All-Star Appearances**: Recognition by peers and selection to All-Star games can indicate a player’s talent level.

8. **Awards**: Any MVPs or other major awards can bolster a player's case.

9. **Postseason Per

### Step 6 - Make Hall of Fame Decision - Only using a Machine Learning Model

> "The goal of probability is not to make precise predictions, but to make more accurate predictions than you would by chance."
>
> -- <cite>John Allen Paulos (famous mathmatecian and author)</cite> 

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame. **Notice the instruction in the prompt is to only use the Machine Learning probability model.** While there might be multiple functions available to the GenAI model via the imported plugin, the instruction is to use only the single "tool" at it's disposal.  

In [6]:
// Use Machine Learning Prediction to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the Hall of Fame? 
Only use the machine learning probability for Hall Of Fame Induction to make your decision.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the machine learning probability for Hall of Fame induction, Mike Trout has a probability of approximately 69.91%. Given this strong probability, I would recommend that he should make the Hall of Fame.

### Step 7 - Make Hall of Fame Decision - Using all of the Available Tools (functions)  

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame. **Notice the instruction in the prompt is to use all of the tools available.** This instructs the GenAI model to gather the intelligence from multiple "expert" areas and combine them to arrive at a decision recommendation.  

In [7]:
// Use all of the available tools (functions) to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Use all of the tools available at your disposal to arrive at a comprehensive decision.
You must cite all the sources using [number] notation of each URL after the reference in order.
The sources should appear in order on the bottom of the decision.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

### Recommendation on Mike Trout's Hall of Fame Candidacy

Based on Mike Trout's impressive career statistics and the analysis of his Hall of Fame candidacy, it is recommended that he should be inducted into the Hall of Fame once eligible. Here are the key factors supporting this recommendation:

1. **Career Statistics**: 
   - **Years Played**: 12
   - **Batting Average**: .303
   - **Home Runs (HR)**: 350
   - **Runs Batted In (RBI)**: 896
   - **Stolen Bases (SB)**: 204
   - **All-Star Appearances**: 9
   - **Total Player Awards**: 21, including multiple MVP awards.

   These statistics highlight Trout's consistent performance and impact on the game over his career.

2. **Probability of Induction**: 
   - A machine learning model estimates Trout's probability of being inducted into the Hall of Fame at approximately **69.9%**. This high probability reflects his strong candidacy relative to historical benchmarks.

3. **Historical Context**: 
   - Trout's achievements, including being 

### Step 8 - Understanding Auto Function Calling Behavior 

How does Semantic Kernel and the LLM know to call the function? What if there are multiple functions or plugins, how does LLM signal Semantic Kernel which function to call? 

Executing the cell below will illustrate that the invocation of the functions is not magic, but guided by the descriptions of the plugin function descriptions and parameter descriptions. Note that the Search function is described as "Perform a Web Search". This provides a big clue to Semantic Kernel that this a possible tool (function) to use to extract current information.

In [8]:
// Method for printing the functions metadata
private void PrintFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"  Function Name: {func.Name}");
    Console.WriteLine($"  Function Description: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("    Function Parameters:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }

    Console.WriteLine();
}
// Get the functions metadata
var functions = semanticKernel.Plugins.GetFunctionsMetadata();

foreach (KernelFunctionMetadata func in functions)
{
    PrintFunction(func);
}

Plugin: BaseballHallOfFamePlugin
  Function Name: GetPlayerProbabilityOfHallOfFame
  Function Description: Retrieves a Machine Learning probability of a player being Inducted to Hall Of Fame using a the player's baseball statistics.
    Function Parameters:
      - baseballStatistics: The statistics of the baseball player.
        default: ''

Plugin: BaseballHallOfFamePlugin
  Function Name: GetBaseballPlayerStats
  Function Description: Retrieves baseball statistics for a given baseball player from historical data.
    Function Parameters:
      - nameOfBaseballPlayer: The name of the baseball player to search for.
        default: ''

Plugin: BaseballHallOfFamePlugin
  Function Name: GetWebSearchResults
  Function Description: Retrieves the web search results with identified URL sources for a given baseball player.
    Function Parameters:
      - nameOfBaseballPlayer: The name of the baseball player to search for.
        default: ''

